# 第一关

In [1]:
import random
import copy
import time 
import math 

In [2]:
def tabu_solve(inst, start_time):

   #函数功能：随机生成一个输出路径（初始解）

   #子函数：初始化路径
   def initial_route():

       route = []

       #range函数：返回从1-n-27的列表
       unvisited = list(range(n))

       count = n

       #随机生成1-n-27的乱序序列，表示初始解。
       while(count != 0):

           index = random.randint(0, count - 1)

           current = unvisited[index]

           route.append(current)
           
           unvisited.pop(index)

           count -= 1

       return route

   #函数功能：输出路径的目标值
   def cal_distance(route):

       distance = 0.0

       for i in range(n - 1):
           #从1号区域到n-1号区域
           distance += get_edge(i, i+1, route)
       #n-1号区域返回1号区域
       distance += get_edge(n-1, 0, route)

       return distance

   #函数功能：获取两点之间的边距
   def get_edge(index_i, index_j, route):

       if index_i == n : index_i = 0

       if index_j == n : index_j = 0

       return edge[route[index_i]][route[index_j]]

   #函数功能：计算邻域（按Swap算子）
   def cal_neighbor(nid_i, nid_j, route):

       #i，j表示点的编号，index_i、j表示点在路径中的序号

       #.index（n）函数用于返回n在列表中的索引值
       index_i = route.index(nid_i)

       index_j = route.index(nid_j)

       delta = 0

       #去重优化
       if(index_i == index_j - 1 or index_i == index_j + n - 1):

           delta += get_edge(index_i, index_j + 1, route) + get_edge(index_i - 1,index_j, route)

           delta -= get_edge(index_i - 1, index_i, route) + get_edge(index_j,index_j + 1, route)

       elif(index_i == index_j + 1 or index_j == index_i + n -1):

           delta += get_edge(index_j, index_i + 1, route) + get_edge(index_j - 1,index_i, route)

           delta -= get_edge(index_j - 1, index_j, route) + get_edge(index_i,index_i + 1, route)

       else:

           delta += get_edge(index_j, index_i - 1, route) + get_edge(index_j,index_i + 1, route)

           delta += get_edge(index_i, index_j - 1, route) + get_edge(index_i,index_j + 1, route)

           delta -= get_edge(index_i, index_i - 1, route) + get_edge(index_i,index_i + 1, route)

           delta -= get_edge(index_j, index_j - 1, route) + get_edge(index_j,index_j + 1, route)

       return delta

           
   #输出路径
   def output_route(info, route, distance):

       print(info, ', tour:', route, ', distance:', distance)

   

   eplison = 0.000001

   iteration = 500 #最大迭代次数

   n= inst.n #区域数量

   tabu_length= int(n * 0.2) #禁忌长度——这里设置为节点数的20%

   points = inst.points

   dist = inst.dist

   edge = [([0] * n) for i in range(n)] #保存点到点直接的距离，二维表

   for j in range(n):

       for i in range(j + 1, n):

           edge[i][j] = edge[j][i] = dist.get((i,j))

   tabu_list = [([0] * n) for i in range(n)] #定于禁忌表，二维表

   #best用于存储搜索最优目标值，local用于存储搜索当前目标值
   best = float('inf')

   best_route = list()

   local = 0.0

   
   #获取初始解
   ini_route = initial_route()
   #当前满意解
   local = cal_distance(ini_route)

   best = min(best, local)

   
   #初始化最优解
   route = copy.copy(ini_route)

   best_route = copy.copy(ini_route)

   #计算初始邻域
   #初始化邻域为哈希表
   neighbors = dict()

   for i in range(n):

       for j in range(i + 1, n):
           #[i,j:i,j,route]
           neighbors[str(i) + ',' + str(j)] = cal_neighbor(i, j, route)

   

   #搜索开始

   for k in range(iteration):

       sorted_neighbors = sorted(neighbors.items(), key = lambda item :item[1])

       #print('sort_neighbors', sorted_neighbors)

       nid_i = nid_j = 0

       flag = 0

       for neighbor in sorted_neighbors:

           nids = neighbor[0].split(',')

           nid_i = int(nids[0])

           nid_j = int(nids[1])

           delta = neighbor[1]

           temp_local = local + delta

           # 破禁准则
           #最优解
           if(temp_local < best):

                local = temp_local

                best = local

                flag = 1

           else:

                #禁忌表数值非零时，跳过当前邻域
                if(tabu_list[nid_i][nid_j] !=0):

                    continue

                else:

                   local = temp_local

           break

       # 根据上述邻域选择的结果，更新路径（按swap交换两个节点）

       index_i = route.index(nid_i)

       index_j = route.index(nid_j)

       route.pop(index_i)

       route.insert(index_i, nid_j)

       route.pop(index_j)

       route.insert(index_j, nid_i)

       if(flag == 1):

           best_route = copy.copy(route)

       # 更新禁忌表

       for i in range(n):

           for j in range(n - i):

                if(tabu_list[i][j] != 0):

                    tabu_list[i][j] -= 1

       tabu_list[nid_i][nid_j] += tabu_length

       # 更新邻域

       for i in range(n):

           for j in range(i + 1, n):

                neighbors[str(i) + ',' +str(j)] = cal_neighbor(i, j, route)
       
       print("迭代次数",k,"\t当前满意解",best)

   #将TS结果按字典形式返回

   result = dict()

   result['tour'] = str(best_route)

   result['cost'] = best

   return result         

In [3]:
#Instance类的定义方式

class Instance:

   def __init__(self):

       self.n = 30 # 结点数量

       self.points = [
[ 9800,36 ],[ 9600,72 ],[ 9410,101 ],[ 9260,151 ],[ 9070,180 ],
[ 8870,216 ],[ 8720,266 ],[ 8625,295],[ 8525,331 ],[ 8425,367 ],
[ 8275,417 ],[ 8175,453 ],[ 8080,482 ],[ 7980,518 ],
[ 7880,554 ],[ 7780,590 ],[ 7630,640 ],
[ 7480,690 ],[ 7380,726 ],[ 7280,762 ],[ 7185,791 ],[ 7090,820 ],[ 6990,856 ],
[ 7895,885 ],[ 6745,935 ],[ 6645,971 ],[ 6550,1000 ],[ 6455,1029 ],[ 6355,1065 ],[ 6255,1101]

	];
       self.dist = {(i, j) : math.sqrt(sum((self.points[i][k] -self.points[j][k])**2 \

                for k in range(2))) for i in range(self.n) for j in range(i)} # 点之间的距离


In [4]:
def main():

   #初始化算例并输出
   inst = Instance() 

   #开始禁忌搜索
   start_time = time.time()

   tabu_result = tabu_solve(inst, start_time) #调用tabuSearch求解

   end_time = time.time()

   tabu_time = end_time - start_time

   tabu_result['time'] = tabu_time

   #输出结果
   print(tabu_result)

In [5]:
if __name__ == '__main__':
    main()

迭代次数 0 	当前满意解 30293.048397277576
迭代次数 1 	当前满意解 26668.343939490158
迭代次数 2 	当前满意解 23325.335585280533
迭代次数 3 	当前满意解 20203.5783566342
迭代次数 4 	当前满意解 18545.265631746443
迭代次数 5 	当前满意解 16351.980489885964
迭代次数 6 	当前满意解 14273.831617854254
迭代次数 7 	当前满意解 13274.534808355926
迭代次数 8 	当前满意解 12429.615654207944
迭代次数 9 	当前满意解 11791.953964922603
迭代次数 10 	当前满意解 11366.906550334625
迭代次数 11 	当前满意解 11054.141567124163
迭代次数 12 	当前满意解 10855.570145722097
迭代次数 13 	当前满意解 10782.053779146258
迭代次数 14 	当前满意解 10440.10415486521
迭代次数 15 	当前满意解 10425.606637459488
迭代次数 16 	当前满意解 10411.644106454007
迭代次数 17 	当前满意解 10213.063784690696
迭代次数 18 	当前满意解 10212.131290712381
迭代次数 19 	当前满意解 9828.342700804329
迭代次数 20 	当前满意解 9827.915384565995
迭代次数 21 	当前满意解 9827.915384565995
迭代次数 22 	当前满意解 9827.648315547722
迭代次数 23 	当前满意解 9827.448465555271
迭代次数 24 	当前满意解 9615.739235835377
迭代次数 25 	当前满意解 9424.288653501884
迭代次数 26 	当前满意解 9013.58056079854
迭代次数 27 	当前满意解 8698.7034896883
迭代次数 28 	当前满意解 8486.194340502192
迭代次数 29 	当前满意解 8169.994651751478
迭代次数 30

迭代次数 289 	当前满意解 7449.965452627162
迭代次数 290 	当前满意解 7449.965452627162
迭代次数 291 	当前满意解 7449.965452627162
迭代次数 292 	当前满意解 7449.965452627162
迭代次数 293 	当前满意解 7449.965452627162
迭代次数 294 	当前满意解 7449.965452627162
迭代次数 295 	当前满意解 7449.965452627162
迭代次数 296 	当前满意解 7449.965452627162
迭代次数 297 	当前满意解 7449.965452627162
迭代次数 298 	当前满意解 7449.965452627162
迭代次数 299 	当前满意解 7449.965452627162
迭代次数 300 	当前满意解 7449.965452627162
迭代次数 301 	当前满意解 7449.965452627162
迭代次数 302 	当前满意解 7449.965452627162
迭代次数 303 	当前满意解 7449.965452627162
迭代次数 304 	当前满意解 7449.965452627162
迭代次数 305 	当前满意解 7449.965452627162
迭代次数 306 	当前满意解 7449.965452627162
迭代次数 307 	当前满意解 7449.965452627162
迭代次数 308 	当前满意解 7449.965452627162
迭代次数 309 	当前满意解 7449.965452627162
迭代次数 310 	当前满意解 7449.965452627162
迭代次数 311 	当前满意解 7449.965452627162
迭代次数 312 	当前满意解 7449.965452627162
迭代次数 313 	当前满意解 7449.965452627162
迭代次数 314 	当前满意解 7449.965452627162
迭代次数 315 	当前满意解 7449.965452627162
迭代次数 316 	当前满意解 7449.965452627162
迭代次数 317 	当前满意解 7449.965452627162
迭代次数 318 	当前满意

# 第二关


In [6]:
def tabu_solve(inst, start_time):

   #函数功能：随机生成一个输出路径（初始解）

   #子函数：初始化路径
   def initial_route():

       route = []

       #range函数：返回从1-n-64的列表
       unvisited = list(range(n))

       count = n

       #随机生成1-n-64的乱序序列，表示初始解。
       while(count != 0):

           index = random.randint(0, count - 1)

           current = unvisited[index]

           route.append(current)
           
           unvisited.pop(index)

           count -= 1

       return route

   #函数功能：输出路径的目标值
   def cal_distance(route):

       distance = 0.0

       for i in range(n - 1):
           #从1号区域到n-1号区域
           distance += get_edge(i, i+1, route)
       #n-1号区域返回1号区域
       distance += get_edge(n-1, 0, route)

       return distance

   #函数功能：获取两点之间的边距
   def get_edge(index_i, index_j, route):

       if index_i == n : index_i = 0

       if index_j == n : index_j = 0

       return edge[route[index_i]][route[index_j]]

   #函数功能：计算邻域（按Swap算子）
   def cal_neighbor(nid_i, nid_j, route):

       #i，j表示点的编号，index_i、j表示点在路径中的序号

       #.index（n）函数用于返回n在列表中的索引值
       index_i = route.index(nid_i)

       index_j = route.index(nid_j)

       delta = 0

       #去重优化
       if(index_i == index_j - 1 or index_i == index_j + n - 1):

           delta += get_edge(index_i, index_j + 1, route) + get_edge(index_i - 1,index_j, route)

           delta -= get_edge(index_i - 1, index_i, route) + get_edge(index_j,index_j + 1, route)

       elif(index_i == index_j + 1 or index_j == index_i + n -1):

           delta += get_edge(index_j, index_i + 1, route) + get_edge(index_j - 1,index_i, route)

           delta -= get_edge(index_j - 1, index_j, route) + get_edge(index_i,index_i + 1, route)

       else:

           delta += get_edge(index_j, index_i - 1, route) + get_edge(index_j,index_i + 1, route)

           delta += get_edge(index_i, index_j - 1, route) + get_edge(index_i,index_j + 1, route)

           delta -= get_edge(index_i, index_i - 1, route) + get_edge(index_i,index_i + 1, route)

           delta -= get_edge(index_j, index_j - 1, route) + get_edge(index_j,index_j + 1, route)

       return delta

           
   #输出路径
   def output_route(info, route, distance):

       print(info, ', tour:', route, ', distance:', distance)

   

   eplison = 0.000001

   iteration = 500 #最大迭代次数

   n= inst.n #区域数量

   tabu_length= int(n * 0.2) #禁忌长度——这里设置为节点数的20%

   points = inst.points

   dist = inst.dist

   edge = [([0] * n) for i in range(n)] #保存点到点直接的距离，二维表

   for j in range(n):

       for i in range(j + 1, n):

           edge[i][j] = edge[j][i] = dist.get((i,j))

   tabu_list = [([0] * n) for i in range(n)] #定于禁忌表，二维表

   #best用于存储搜索最优目标值，local用于存储搜索当前目标值
   best = float('inf')

   best_route = list()

   local = 0.0

   
   #获取初始解
   ini_route = initial_route()
   #当前满意解
   local = cal_distance(ini_route)

   best = min(best, local)

   
   #初始化最优解
   route = copy.copy(ini_route)

   best_route = copy.copy(ini_route)

   #计算初始邻域
   #初始化邻域为哈希表
   neighbors = dict()

   for i in range(n):

       for j in range(i + 1, n):
           #[i,j:i,j,route]
           neighbors[str(i) + ',' + str(j)] = cal_neighbor(i, j, route)

   

   #搜索开始

   for k in range(iteration):

       sorted_neighbors = sorted(neighbors.items(), key = lambda item :item[1])

       #print('sort_neighbors', sorted_neighbors)

       nid_i = nid_j = 0

       flag = 0

       for neighbor in sorted_neighbors:

           nids = neighbor[0].split(',')

           nid_i = int(nids[0])

           nid_j = int(nids[1])

           delta = neighbor[1]

           temp_local = local + delta

           # 破禁准则
           #最优解
           if(temp_local < best):

                local = temp_local

                best = local

                flag = 1

           else:

                #禁忌表数值非零时，跳过当前邻域
                if(tabu_list[nid_i][nid_j] !=0):

                    continue

                else:

                   local = temp_local

           break

       # 根据上述邻域选择的结果，更新路径（按swap交换两个节点）

       index_i = route.index(nid_i)

       index_j = route.index(nid_j)

       route.pop(index_i)

       route.insert(index_i, nid_j)

       route.pop(index_j)

       route.insert(index_j, nid_i)

       if(flag == 1):

           best_route = copy.copy(route)

       # 更新禁忌表

       for i in range(n):

           for j in range(n - i):

                if(tabu_list[i][j] != 0):

                    tabu_list[i][j] -= 1

       tabu_list[nid_i][nid_j] += tabu_length

       # 更新邻域

       for i in range(n):

           for j in range(i + 1, n):

                neighbors[str(i) + ',' +str(j)] = cal_neighbor(i, j, route)
       
       print("迭代次数",k,"\t当前满意解",best)

   #将TS结果按字典形式返回

   result = dict()

   result['tour'] = str(best_route)

   result['cost'] = best

   return result         

In [7]:
if __name__ == '__main__':
    main()

迭代次数 0 	当前满意解 33606.68922177426
迭代次数 1 	当前满意解 29324.27229961565
迭代次数 2 	当前满意解 25262.742321118534
迭代次数 3 	当前满意解 22404.36754665467
迭代次数 4 	当前满意解 20552.542498198047
迭代次数 5 	当前满意解 18856.908709848896
迭代次数 6 	当前满意解 17938.608812171955
迭代次数 7 	当前满意解 17130.391925924705
迭代次数 8 	当前满意解 16692.791720808855
迭代次数 9 	当前满意解 16481.437400898474
迭代次数 10 	当前满意解 16290.558349002898
迭代次数 11 	当前满意解 16284.076549490419
迭代次数 12 	当前满意解 16281.473344554684
迭代次数 13 	当前满意解 16279.82099785393
迭代次数 14 	当前满意解 16279.78033712395
迭代次数 15 	当前满意解 16279.748445481353
迭代次数 16 	当前满意解 16279.724807659697
迭代次数 17 	当前满意解 15965.267621959438
迭代次数 18 	当前满意解 15582.122136283107
迭代次数 19 	当前满意解 15582.01311210619
迭代次数 20 	当前满意解 15581.109114963048
迭代次数 21 	当前满意解 15581.033506751583
迭代次数 22 	当前满意解 15375.59756265426
迭代次数 23 	当前满意解 15251.752207904015
迭代次数 24 	当前满意解 15237.341046484791
迭代次数 25 	当前满意解 14880.449428814009
迭代次数 26 	当前满意解 14667.911347284837
迭代次数 27 	当前满意解 14555.42411200036
迭代次数 28 	当前满意解 14239.277432593393
迭代次数 29 	当前满意解 14135.63902806883

迭代次数 333 	当前满意解 7449.965452627177
迭代次数 334 	当前满意解 7449.965452627177
迭代次数 335 	当前满意解 7449.965452627177
迭代次数 336 	当前满意解 7449.965452627177
迭代次数 337 	当前满意解 7449.965452627177
迭代次数 338 	当前满意解 7449.965452627177
迭代次数 339 	当前满意解 7449.965452627177
迭代次数 340 	当前满意解 7449.965452627177
迭代次数 341 	当前满意解 7449.965452627177
迭代次数 342 	当前满意解 7449.965452627177
迭代次数 343 	当前满意解 7449.965452627177
迭代次数 344 	当前满意解 7449.965452627177
迭代次数 345 	当前满意解 7449.965452627177
迭代次数 346 	当前满意解 7449.965452627177
迭代次数 347 	当前满意解 7449.965452627177
迭代次数 348 	当前满意解 7449.965452627177
迭代次数 349 	当前满意解 7449.965452627177
迭代次数 350 	当前满意解 7449.965452627177
迭代次数 351 	当前满意解 7449.965452627177
迭代次数 352 	当前满意解 7449.965452627177
迭代次数 353 	当前满意解 7449.965452627177
迭代次数 354 	当前满意解 7449.965452627177
迭代次数 355 	当前满意解 7449.965452627177
迭代次数 356 	当前满意解 7449.965452627177
迭代次数 357 	当前满意解 7449.965452627177
迭代次数 358 	当前满意解 7449.965452627177
迭代次数 359 	当前满意解 7449.965452627177
迭代次数 360 	当前满意解 7449.965452627177
迭代次数 361 	当前满意解 7449.965452627177
迭代次数 362 	当前满意